In [ ]:
import os
os.chdir('models/efficientdet/')
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from src.dataset import WaymoDataset, Resizer, Normalizer
wd = WaymoDataset(cameras=['FRONT'], scope='validation', transform=transforms.Compose([Normalizer(), Resizer()]))
model = torch.load('trained_models/signatrix_efficientdet_coco.pth',
          map_location=torch.device('cpu')).module

In [ ]:
import os
import argparse
import torch
from torchvision import transforms
from src.dataset import CocoDataset, Resizer, Normalizer
from src.config import COCO_CLASSES, colors
import cv2
import shutil
import tensorflow.compat.v1 as tf

dataset = wd
model = ed

for index in range(1):
    data = dataset[index]
    scale = data['scale']
    with torch.no_grad():
        scores, labels, boxes = model(data['img'].permute(2, 0, 1).float().unsqueeze(dim=0))
        boxes /= scale
    if boxes.shape[0] > 0:
        
        output_image = tf.image.decode_jpeg(data['raw']["image"]).numpy()
#         output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
        for box_id in range(boxes.shape[0]):
            pred_prob = float(scores[box_id])
            if pred_prob < 0.3:
                break
            pred_label = int(labels[box_id])
            xmin, ymin, xmax, ymax = boxes[box_id, :]
            color = colors[pred_label]
            cv2.rectangle(output_image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 15)
            text_size = cv2.getTextSize(COCO_CLASSES[pred_label] + ' : %.2f' % pred_prob, cv2.FONT_HERSHEY_PLAIN, 2, 1)[0]

#             cv2.rectangle(output_image, (xmin, ymin), (xmin + text_size[0] +100, ymin + text_size[1] - 100), colors[pred_label+1], -1)
            cv2.putText(
                output_image, COCO_CLASSES[pred_label] + ' : %.2f' % pred_prob,
                (xmin, ymin + text_size[1] - 40), cv2.FONT_HERSHEY_PLAIN, 5,
                (255, 255, 255), 3)

        plt.imshow( output_image)
        plt.show()


In [ ]:
import os
import argparse
import torch

import cv2
import shutil
%matplotlib inline
from matplotlib import pyplot as plt
import tensorflow.compat.v1 as tf
x = tf.image.decode_jpeg(wd[0]['raw']["image"]).numpy()
plt.imshow(x)
plt.axis('off')

plt.savefig('fig.jpg',bbox_inches='tight',transparent=True, pad_inches=0)
print(x.shape)

In [ ]:
output_image = x
# output_image = cv2.cvtColor(output_image)
labels = wd[0]['raw']['labels'].labels
for box_id in range(len(labels)):
    label = labels[box_id]
    box = label.box
    pred_label = label.type
    xy1=(label.box.center_x - 0.5 * label.box.length,
                label.box.center_y - 0.5 * label.box.width)
    xy2=(label.box.center_x + 0.5 * label.box.length,
                label.box.center_y + 0.5 * label.box.width)
    xmin, ymin = (int(i) for i in xy1)
    xmax, ymax = (int(i) for i in xy2)
    color = colors[pred_label]
    cv2.rectangle(output_image, (xmin, ymin), (xmax, ymax), color, 2)
    text_size = cv2.getTextSize(str(pred_label) + ' : %.2f' % pred_prob, cv2.FONT_HERSHEY_PLAIN, 1, 1)[0]

    cv2.rectangle(output_image, (xmin, ymin), (xmin, ymin), color, -1)
#     cv2.putText(
#         output_image, COCO_CLASSES[pred_label] + ' : %.2f' % pred_prob,
#         (xmin, ymin + text_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1,
#         (255, 255, 255), 1)
plt.imshow( output_image)


In [ ]:
os.chdir('/home/project_x/evaluation/waymo-od/waymo_open_dataset')


In [ ]:
# wd[5]['raw']['context']
debox = lambda x: [x.center_x, x.center_y, x.width, x.length]
frame = wd[5]['raw']
labels = frame['labels'].labels
num_boxes = len(labels)
pred_frame_id = [
    frame['context']['name']+ frame[
        'context'][
        'time_frame_idx'] for _ in range(num_boxes)
                ]
pred_bbox = [debox(i.box) for i in labels]
pred_type = [i.type for i in labels]
pred_score = [1 for i in range(num_boxes)]

In [ ]:
[i for i, s in enumerate(wd.filepaths) if 'blob_130' in s] # 0,105,167


In [ ]:
import torch
from waymo_open_dataset.protos import metrics_pb2
from waymo_open_dataset import label_pb2,dataset_pb2

# ['vehicle', 'pedestrian', 'sign', 'cyclist']
# Could have included things like boat, train, airplane (vehicles?)
class_mapper = {
    0: 2, # person
    1:4, # in COCO it's bicycle not cyclist...
    2:1, # car
    3:1, # motorcycle
    5:1, # bus
    7:1, # truck
    11:3 # stop sign
}
score_threshold = 0.4 
def get_preds(sample, model):
    with torch.no_grad():
        scores, annot, boxes = model(sample['img'].permute(2, 0, 1).float().unsqueeze(dim=0))
        boxes /= sample['scale']
    return scores, annot, boxes
def convert_to_waymo_boxes(boxes):
    tb = boxes.T
    length = tb[2]-tb[0]
    width = tb[3]-tb[1]
    tb[0] = tb[2]-length # center x
    tb[1] = tb[3]-width # center y
    tb[2] = length
    tb[3] = width
    res = tb.T
    return res
def gt_map(o, data):
    box = label_pb2.Label.Box()
    box.center_x = data.box.center_x
    box.center_y = data.box.center_y
    box.length = data.box.length
    box.width = data.box.width

    o.object.box.CopyFrom(box)
    o.score = 1
    # Use correct type.
    o.object.type = data.type
def pred_map(o, data):
    s, c, b = data
    box = label_pb2.Label.Box()
    box.center_x = b[0]
    box.center_y = b[1]
    box.length = b[2]
    box.width = b[3]
    o.object.box.CopyFrom(box)
    o.score = s
    # Use correct type.
    o.object.type = class_mapper[c.item()]
def filter_preds(data):
    scores, classes, boxes = data
    valid_idxs = []
    for c,cls in enumerate(classes):
        if (cls in [0,1,2,3, 5,7,11])*(scores[c]>score_threshold):
            valid_idxs.append(c)
    valid_idxs = torch.tensor(valid_idxs).long()
    scores = scores.index_select(0, valid_idxs)
    classes = classes.index_select(0, valid_idxs)
    boxes = boxes.index_select(0, valid_idxs)
    return (scores, classes, boxes)
def process_frame_objects(objects, data, name, ts, cam, scope):
    if scope=='preds':
        data = filter_preds(data)
        num_objects = len(data[0])
        boxes = convert_to_waymo_boxes(data[2])
    else:
        num_objects = len(data)
    for c in range(num_objects):
        o = metrics_pb2.Object()
        o.context_name = name
        o.frame_timestamp_micros = ts
        o.camera_name = cam = cam
        if scope=='preds':
            data_i = (data[0][c], data[1][c], boxes[c]) # scores, classes, boxes
            pred_map(o, data_i)
        else:
            gt_map(o, data[c])
        objects.objects.append(o)

def create_pd_file(dataset, model): # preds or gt
    """Creates a prediction objects file."""
    gt_objects = metrics_pb2.Objects()
    pred_objects = metrics_pb2.Objects()
    for c, item in enumerate(dataset):
        preds = get_preds(item, model)
        ctx = item['raw']['context']
        cam = getattr(dataset_pb2.CameraName, dataset.get_cam_type(c))
        # Populate preds
        process_frame_objects(pred_objects, preds, ctx['name'],
                              ctx['timestamp'], cam, 'preds')
        process_frame_objects(gt_objects, item['raw']['labels'].labels, ctx['name'],
                              ctx['timestamp'], cam, 'gt')        
        if c%2==0:
            print(f'Processed {c} frames out of {len(dataset)}')
        if c==10:
            break
            
    # Write objects to a file.
    f = open('/home/project_x/evaluation/results/preds.bin', 'wb')
    f.write(pred_objects.SerializeToString())
    f.close()
    f = open('/home/project_x/evaluation/results/gt.bin', 'wb')
    f.write(gt_objects.SerializeToString())
    f.close()
create_pd_file(wd, model)

In [ ]:
'done'

In [ ]:
boxes.index_select(0, torch.tensor([5,3,10, 2]))

In [ ]:
dataset[0]['raw']['labels'].labels[0]

In [ ]:
COCO_CLASSES

In [ ]:
# from waymo_open_dataset import metrics_pb2
# y = label_pb2.Label
# y.TYPE_CYCLIST
from waymo_open_dataset import dataset_pb2


In [ ]:
getattr(dataset_pb2.CameraName, wd.get_cam_type(1))

In [ ]:
torch.tensor([5]).long()

In [ ]:
# bazel-bin/waymo_open_dataset/metrics/tools/compute_detection_metrics_main /home/project_x/evaluation/results/preds.bin  /home/project_x/evaluation/results/gt.bin
y = label_pb2.Label
dir(y.Box)


In [ ]:
label_pb2.Label.TYPE_CYCLIST